# By the end of this exercise, you should have a file named acquire.py that contains the specified functions. If you wish, you may break your work into separate files for each website (e.g. acquire_codeup_blog.py and acquire_news_articles.py), but the end function should be present in acquire.py (that is, acquire.py should import get_blog_articles from the acquire_codeup_blog module.)

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup as bs

##  1. Scrape Codeup's Blog and record the urls for at least 5 distinct blog posts. For each post, you should scrape at least the post's title and content.

## Encapsulate your work in a function named get_blog_articles that will return a list of dictionaries, with each dictionary representing one article. The shape of each dictionary should look like this:

In [4]:
headers = {'User-Agent': 'Codeup Data Science'}

In [2]:
# {
#     'title': 'the title of the article',
#     'content': 'the full text content of the article'
# }

In [6]:
r = requests.get('https://codeup.com/blog/', headers = headers)

In [9]:
# Make a soup variable holding the response content
soup = bs(r.content, 'html.parser')

In [24]:
#creates a random array of 5 different integers between 1 and 15 (the size of the nuber of articles).  Then subtract 1 from 
a = range(15)
blog_posts = np.random.choice(a, size=5, replace=False, p=None)
blog_posts = blog_posts
blog_posts = list(blog_posts)

In [ ]:
# Use the arrays to go into a specific blog post, then from there grab the details we want:
# h1 class = entry-title gets the title
# span class = published for date published
# div class "entry-content"

In [27]:
blog_list = soup.find_all('h2', attrs={'class':'entry-title'})
blog_list

[<h2 class="entry-title"><a href="https://codeup.com/featured/what-jobs-can-you-get-after-a-coding-bootcamp-part-2-cloud-administration/">What Jobs Can You Get After a Coding Bootcamp? Part 2: Cloud Administration</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/data-science/jobs-after-a-coding-bootcamp-part-1-data-science/">What Jobs Can You Get After a Coding Bootcamp? Part 1: Data Science</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/workshops/san-antonio/in-person-workshop-learn-to-code-javascript-on-7-26/">In-Person Workshop: Learn to Code – JavaScript on 7/26</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/workshops/in-person-workshop-learn-to-code-python-on-7-19/">In-Person Workshop: Learn to Code – Python on 7/19</a></h2>,
 <h2 class="entry-title"><a href="https://codeup.com/workshops/dallas/free-javascript-workshop-at-codeup-dallas-on-6-28/">Free JavaScript Workshop at Codeup Dallas on 6/28</a></h2>,
 <h2 class="entry-title"><a hre

In [42]:
blog_links = []
for n in blog_posts:
    blog = blog_list[n]
    blog_links.append(blog.a["href"])
blog_links

['https://codeup.com/dallas-newsletter/codeup-dallas-joins-career-day/',
 'https://codeup.com/featured/what-jobs-can-you-get-after-a-coding-bootcamp-part-2-cloud-administration/',
 'https://codeup.com/workshops/in-person-workshop-learn-to-code-python-on-7-19/',
 'https://codeup.com/workshops/codeup-dallas-how-to-succeed-at-a-coding-bootcamp-on-june-9th/',
 'https://codeup.com/data-science/jobs-after-a-coding-bootcamp-part-1-data-science/']

In [50]:
# Now using the blog direct urls, build the df 
blogs_info = []
for blog in blog_links:
    blog_info = {}
    r = requests.get(blog, headers = headers)
    soup = bs(r.content, 'html.parser')
    blog_info['Title'] = soup.find('h1', attrs = {'class':'entry-title'}).text
    blog_info['Published'] = soup.find('span', attrs = {'class' : 'published'}).text
    blog_info['Content'] = soup.find('div', attrs = {'class' : 'entry-content'}).text.strip()
    blogs_info.append(blog_info)
pd.DataFrame(blogs_info).set_index('Title')   

,Published,Content
Title,,
Codeup Dallas Joins Career Day,"May 16, 2022",Last week our Codeup Dallas team participated ...
What Jobs Can You Get After a Coding Bootcamp? Part 2: Cloud Administration,"Jul 14, 2022",Have you been considering a career in Cloud Ad...
In-Person Workshop: Learn to Code – Python on 7/19,"Jun 20, 2022","According to LinkedIn, the “#1 Most Promising ..."
Codeup Dallas: How to Succeed at a Coding Bootcamp on June 9th,"May 23, 2022",This event is the perfect opportunity for peop...
What Jobs Can You Get After a Coding Bootcamp? Part 1: Data Science,"Jul 7, 2022",If you are interested in embarking on a career...


In [51]:
from acquire import acquire_codeup_blog

In [52]:
acquire_codeup_blog()

[{'Title': 'Codeup Dallas Joins Career Day',
  'Published': 'May 16, 2022',
  'Content': 'Last week our Codeup Dallas team participated in Career Day at James Madison High School which also included students from Lincoln High School. This Career Day was an initiative by United Way Dallas. Toyota, Goodwill, Dallas college, and the US Army we’re all in attendance as well to support the Career Day.\xa0\nOur team directly impacted about 60 students! We held three 45 minute sessions where we talked about the following:\xa0\n\nNon-traditional career paths\xa0\nThe different careers in tech\xa0\nProvided resources for creating a solid first resume\xa0\nDiscussed what we each did at Codeup\xa0\n\nWe wrapped up each session with a fun Kahoot review and awarded some Codeup swag to the top winners!\xa0\nWe want to thank United Way Dallas for having us out to support our local Dallas community. If you would like our Coduep team to come out and talk to your students about non-traditional career pat

## 2. We will now be scraping text data from inshorts, a website that provides a brief overview of many different topics.

Write a function that scrapes the news articles for the following topics:

- Business
- Sports
- Technology
- Entertainment
- The end product of this should be a function named get_news_articles that returns a list of dictionaries, where each dictionary has this shape:

In [53]:
# {
#     'title': 'The article title',
#     'content': 'The article content',
#     'category': 'business' # for example
# }

In [60]:
# Create a target list of pages to go to to scrape relavent data
target_list = ['business','sports','technology','entertainment']
base_url = 'https://inshorts.com/en/read/'

In [61]:
target_list = [(base_url + target) for target in target_list]

In [99]:
for x in soup.find_all('div', attrs = {'class':'news-card z-depth-1'}):
    print(x)
    print('********')
    print(x.find('div', attrs = {'itemprop' : 'articleBody'}).text.strip())
    print('********')
    print('\n----------\n----------\n')

<div class="news-card z-depth-1" itemscope="" itemtype="http://schema.org/NewsArticle">
<span content="" itemid="https://inshorts.com/en/news/it-doesnt-bother-me-if-somebody-bitches-about-me-ranbir-on-being-called-a-casanova-1657964443718" itemprop="mainEntityOfPage" itemscope="" itemtype="https://schema.org/WebPage"></span>
<span itemprop="author" itemscope="itemscope" itemtype="https://schema.org/Person">
<span content="Daisy Mowke" itemprop="name"></span>
</span>
<span content="It doesn't bother me if somebody bitches about me: Ranbir on being called a casanova" itemprop="description"></span>
<span itemprop="image" itemscope="" itemtype="https://schema.org/ImageObject">
<meta content="https://static.inshorts.com/inshorts/images/v1/variants/jpg/m/2022/07_jul/16_sat/img_1657963082043_19.jpg?" itemprop="url"/>
<meta content="864" itemprop="width"/>
<meta content="483" itemprop="height"/>
</span>
<span itemprop="publisher" itemscope="itemscope" itemtype="https://schema.org/Organization"

In [102]:
blogs_info = []
for url in target_list:
    r = requests.get(url)
    soup = bs(r.content, 'html.parser')
    for blog in soup.find_all('div', attrs = {'class':'news-card z-depth-1'}):
        blog_info = {}
        blog_info['title'] = blog.find('span', attrs = {'itemprop':'headline'}).text
        blog_info['content'] = blog.find('div', attrs = {'itemprop' : 'articleBody'}).text.strip()
        blog_info['category'] = soup.find('title').text.split(' ')[0]
        blogs_info.append(blog_info)

In [103]:
blogs_info

[{'title': 'Rupee edges closer to 80 per US dollar, opens at record low of 79.94',
  'content': "The rupee on Friday opened at a record low of 79.94 against the US dollar. So far this year, the US dollar is up 7% against the Indian currency. On Thursday, the currency had closed at another record low of 79.90 against the US dollar. Meanwhile, BSE Sensex jumped 316 points to 53,733 in Friday's early session.",
  'category': 'Business'},
 {'title': "World's biggest NFT marketplace OpenSea fires 20% staff citing 'crypto winter'",
  'content': 'The world\'s first and biggest NFT marketplace OpenSea has fired 20% of its employees citing "cryptocurrency winter" and "broad macroeconomic instability". CEO Devin Finzer shared the news on Twitter and said the affected employees will be provided with a generous severance and healthcare coverage into 2023. He added that the company will also help in the placement of these employees.',
  'category': 'Business'},
 {'title': 'Google to slow hiring for

In [104]:
from acquire import get_news_articles

NameError: name 'acquire' is not defined